## Annotation
Annotations in Java serve multiple purposes:
- used by compiler to detect errors / suppress warnings
- used by software tools during compilation to generate code, etc.For example Lombok
- used during runtime

## Creating annotations
To create annotation:

In [1]:
public @interface Information{
    // String value();
    String author();
    String date();
    
    // optional annotation metadata
    int minVersion() default 1;
    String[] reviewers() default {};
}

`Class`, `String`, enums, primitives and arrays are the allowed annotation parameter types. All the elements defined in an annotation are required to be passed while using the array. For example, in the above case writing `@Information(author="John Doe")` is an error because the value of `date` is not provided.

`value` is a special parameter. If it is the only parameter being used then we can just write `@SomeAnnotation(5)` instead of `@SomeAnnotation(value=5)`.

### Annotations of Annotation
`@Documented` annotation can be used to indicate that Javadocs for the class, etc should contain this annotation.

What the annotation can be used with can be specified using `@Target` annotation. Possible values:
- `ElementType.TYPE` : Class, Interface or Enum
- `ElementType.FIELD`
- `ElementType.METHOD`
- `ElementType.PARAMETER` : Method parameter
- `ElementType.CONSTRUCTOR`
- `ElementType.LOCAL_VARIABLE`
- `ElementType.ANNOTATION_TYPE` : Another annotation
- `ElementType.PACKAGE`

For the above Information annotation example, we specify the following target:

In [ ]:
@Target(value = {ElementType.TYPE, ElementType.ANNOTATION_TYPE})
public @interface Information {
    // ...
}

How long the annotation should be kept is determined by `@Retention` annotation. Possible values are:
- `RetentionPolicy.SOURCE` : Discard during the compile. These annotations don't make any sense after the compile has completed, so they aren't written to the bytecode. Example: `@Override`, `@SuppressWarnings`
- `RetentionPolicy.CLASS` : Discard during class load. Useful when doing bytecode-level post-processing. Somewhat surprisingly, this is the default.
- `RetentionPolicy.RUNTIME` : Do not discard. The annotation should be available for reflection at runtime. Example: `@Deprecated`

`@Inherited` annotation indicates that the annotation type can be inherited from the super class. When the user queries the annotation type and the class has no annotation for this type, the class' superclass is queried for the annotation type. This annotation applies only to class declarations.

`@Repeatable` annotation, introduced in Java SE 8, indicates that the marked annotation can be applied more than once to the same declaration or type use. 

### Advanced Usage
Before the Java SE 8 release, annotations could only be applied to declarations. From Java 8 onwards, the place where Annotations can be used have been expanded. The enum `ElementType` now also has the following additional instances:
- `ElementType.TYPE_PARAMETER` : Since Java 8. Indicates that the annotation can be written on the declaration of a type variable
- `ElementType.TYPE_USE` : Since Java 8. Indicates that the annotation can be written on the use of a type (e.g., types appearing in declarations, generics, and casts)
- `ElementType.MODULE`: since Java 9
- `ElementType.RECORD_COMPONENT`: since Java 16

Some examples:

In [ ]:
List<@NonNull String> strings;
graph = (@Immutable Graph) graph;

## Annotation and Reflection
Annotations with retention policy as runtime can be extracted and its properties read at runtime. For example, if we change the retention policy of Information annotation to runtime,

In [ ]:
public class AnnotationDemo {
    public static void main(String[] args) {
        // Get all annotations directly applieed to the given element
        Annotation[] annotations = Demo.class.getDeclaredAnnotations();
        for (Annotation annotation : annotations) {
            if (annotation instanceof Information) {
                System.out.println("Author: " + ((Information) annotation).author());
                System.out.println("Date: " + ((Information) annotation).date());
            }
        }
    }
}

@Information(author = "John Doe", date = "12-12-2020")
class Demo {}

The above code can be shortened to:

In [ ]:
public static void main(String[] args) {
    if(Demo.class.isAnnotationPresent(Information.class)){
        Information info = Demo.class.getAnnotation(Information.class);
        System.out.println("Author: " + info.author());
        System.out.println("Date: " + info.date());
    }
}